# Create a database called 'Wine'

# Import modules

In [46]:
import pandas as pd 

# Import a table for csv

In [47]:
raw = pd.read_csv("dataset/winemag-data_first150k.csv").drop("Unnamed: 0", axis=1)
print("This dataset has: {} columns {} rows".format(raw.shape[0], raw.shape[1]))

This dataset has: 150930 columns 10 rows


In [48]:
raw.head()

,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


# Check for missing values

In [49]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150930 entries, 0 to 150929
Data columns (total 10 columns):
country        150925 non-null object
description    150930 non-null object
designation    105195 non-null object
points         150930 non-null int64
price          137235 non-null float64
province       150925 non-null object
region_1       125870 non-null object
region_2       60953 non-null object
variety        150930 non-null object
winery         150930 non-null object
dtypes: float64(1), int64(1), object(8)
memory usage: 11.5+ MB


# Check for any duplicate rows in description

In [50]:
descrip_unique_row = len(raw['description'].unique())

In [51]:
wine_duplicates = raw[raw.duplicated('description')]
wine_duplicates.head()

,country,description,designation,points,price,province,region_1,region_2,variety,winery
300,US,This standout Rocks District wine brings earth...,The Funk Estate,94,60.0,Washington,Walla Walla Valley (WA),Columbia Valley,Syrah,Saviah
305,New Zealand,"Yields were down in 2015, but intensity is up,...",Maté's Vineyard,94,57.0,Kumeu,NaN,NaN,Chardonnay,Kumeu River
420,US,"Though just four years old, this wine seems ad...",NaN,87,36.0,Washington,Columbia Valley (WA),Columbia Valley,Cabernet Sauvignon,Dussek Family Cellars
422,US,Tawny in color and wafting of potpourri and fa...,NaN,87,20.0,New York,Finger Lakes,Finger Lakes,Cabernet Franc,Serenity
423,US,"The aromas on this wine are quite light, conve...",Weinbau,87,32.0,Washington,Wahluke Slope,Columbia Valley,Grenache,Sol Stone


In [52]:
duplicate_example = wine_duplicates['description'].iloc[0]

In [53]:
raw[raw['description']== duplicate_example]

,country,description,designation,points,price,province,region_1,region_2,variety,winery
29,US,This standout Rocks District wine brings earth...,The Funk Estate,94,60.0,Washington,Walla Walla Valley (WA),Columbia Valley,Syrah,Saviah
300,US,This standout Rocks District wine brings earth...,The Funk Estate,94,60.0,Washington,Walla Walla Valley (WA),Columbia Valley,Syrah,Saviah


As we can see above, some of the columns are duplicates. Before uploading the data to the database, we want to remove all duplicates. 

In [54]:
wine = raw.drop_duplicates(subset='description')
print("This dataset has: {} columns {} rows".format(wine.shape[0], wine.shape[1]))

This dataset has: 97821 columns 10 rows


In [68]:
def unique_item_to_df(origin_df, col_name):
    df_to_list = origin_df[col_name].unique().tolist()
    return(pd.DataFrame({'col_name': df_to_list}))

In [140]:
unique_country_df = unique_item_to_df(wine, 'country')
unique_designation_df = unique_item_to_df(wine, 'designation')
unique_province_df = unique_item_to_df(wine, 'province')
unique_region1_df = unique_item_to_df(wine, 'region_1')
unique_region2_df = unique_item_to_df(wine, 'region_2')
unique_variety_df = unique_item_to_df(wine, 'variety')
unique_winery_df = unique_item_to_df(wine, 'winery')

# Interact with Database Using psychopg2

## Import package for connecting to database

Pip Install
https://www.a2hosting.ca/kb/developer-corner/postgresql/connecting-to-postgresql-using-python

https://stackoverflow.com/questions/33866695/install-psycopg2-on-mac-osx-10-9-5

In terminal:
* pip install psycopg2-binary

## Import Package After Installing

In [22]:
import psycopg2

## Create Connection

In [105]:
DB_NAME = "wine"
DB_USER = "postgres"
DB_HOST = "localhost"
DB_PASS = open("password.txt", "r").read()
DB_PORT = "5432"

In [91]:
try:
    conn = psycopg2.connect(database = DB_NAME, 
                            user = DB_USER, 
                            password = DB_PASS, 
                            host = DB_HOST, 
                            port = DB_PORT)
    print("Database opened successfully")

except:
    print("Database not connected")

Database opened successfully


## Create a Cursor Object

https://www.datacamp.com/community/tutorials/tutorial-postgresql-python



https://www.youtube.com/watch?v=2PDkXviEMD0

* initial connection
* select
* insert

In [92]:
cursor = conn.cursor()

Checking if a certain table exists in the database:

In [93]:
cursor.execute("select exists(select * from information_schema.tables where table_name=%s)", ('province',))
cursor.fetchone()[0]


False

Creating function that will be called to write all the tables in the database

In [94]:
def check_table_exist(tablename):
    
    cursor.execute("select exists(select * from information_schema.tables where table_name=%s)", 
                   (tablename,))
    
    return(cursor.fetchone()[0]) # Return a boolean

In [80]:
def post_data(connection, table_to_post, tablename):
    
    # check if the table already exists in the database
    
    if check_table_exist(tablename):
        cursor.execute("DROP TABLE " + tablename + " CASCADE")
    
    cursor.execute("CREATE TABLE " + tablename)

## Create Table Individually

In [95]:
cursor.execute("""
CREATE TABLE winery (
winery text
)
""")

In [97]:
conn.commit()

In [99]:
check_table_exist('winery')

True

## Close Cursor

In [85]:
cursor.close()

## Closing Connection

In [86]:
conn.close()

# SQL Alchemy

## import package

In [102]:
from sqlalchemy import create_engine

## Create Engine 

In [142]:
engine = create_engine('postgresql://{}:{}@{}:{}/wine'.format(DB_USER,
                                                              DB_PASS,
                                                              DB_HOST,
                                                              DB_PORT))

## Import DF to SQL table

### Define a function to check if a table already exists

In [158]:
def check_table_exist(tablename, engine):
    check = engine.execute("""
    SELECT EXISTS 
    (SELECT * FROM information_schema.tables 
    WHERE table_name = %s
    )""", 
                           (tablename,))
    return(check.fetchone()[0])

In [188]:
check_table_exist('designation', engine)

False

### Define a function post_data

this function will:

   1) Call check_table_exist. If true, drop table
   
   2) Create a new table

In [180]:
def post_data(tablename, engine, dataframe):
    if check_table_exist(tablename, engine):
        
        print('table already exists!')
        
        engine.execute("DROP TABLE {} CASCADE".format(tablename))
        
        print('old table dropped!')
    else:
        print('table does not exists!')
    
    dataframe.to_sql(tablename, con = engine)
    print('new table created!')

In [186]:
post_data('variety', engine, unique_variety_df)

new table created!


In [187]:
engine.execute("SELECT * FROM variety").fetchall()

[(0, 'Cabernet Sauvignon'),
 (1, 'Tinta de Toro'),
 (2, 'Sauvignon Blanc'),
 (3, 'Pinot Noir'),
 (4, 'Provence red blend'),
 (5, 'Friulano'),
 (6, 'Tannat'),
 (7, 'Chardonnay'),
 (8, 'Tempranillo'),
 (9, 'Malbec'),
 (10, 'Rosé'),
 (11, 'Tempranillo Blend'),
 (12, 'Syrah'),
 (13, 'Mavrud'),
 (14, 'Sangiovese'),
 (15, 'Sparkling Blend'),
 (16, 'Rhône-style White Blend'),
 (17, 'Red Blend'),
 (18, 'Mencía'),
 (19, 'Palomino'),
 (20, 'Petite Sirah'),
 (21, 'Riesling'),
 (22, 'Cabernet Sauvignon-Syrah'),
 (23, 'Portuguese Red'),
 (24, 'Nebbiolo'),
 (25, 'Pinot Gris'),
 (26, 'Meritage'),
 (27, 'Baga'),
 (28, 'Glera'),
 (29, 'Malbec-Merlot'),
 (30, 'Merlot-Malbec'),
 (31, 'Ugni Blanc-Colombard'),
 (32, 'Viognier'),
 (33, 'Cabernet Sauvignon-Cabernet Franc'),
 (34, 'Moscato'),
 (35, 'Pinot Grigio'),
 (36, 'Cabernet Franc'),
 (37, 'White Blend'),
 (38, 'Monastrell'),
 (39, 'Gamay'),
 (40, 'Zinfandel'),
 (41, 'Greco'),
 (42, 'Barbera'),
 (43, 'Grenache'),
 (44, 'Rhône-style Red Blend'),
 (45, 'A